In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("ELA_data_2013_2023.csv")

### **Data Feasibility Review & Data Cleaning/Preprocessing**

The initial data contains 626,462 rows and 18 columns. After filtering for years 2018, 2019, and 2022, the data contains 220,443 rows. Almost all columns do not have missing valuesm with the exception for school name. This is due to the data containing citywide and district level metrics, so school name is not applicable. 

The data was filtered to contain only years surrounding the COVID-19 pandemic, as this is the time period of interest. Unfortunately, data was not given for the years 2020 and 2021, due to cancelled and optional testing, respectively. All scores and levels (number and percentages) were converted to floats. The year was converted to an object. Groups with five or less tested students were suppressed with a value "s". Within this data, 58,872 rows are missing observations for scores and level breakdowns. These observations have been omitted.

In [3]:
from data_cleaning import cleaning_data, change_variable_type
school_data = cleaning_data(data) # filter for year, rename some vars

In [9]:
# school_data.info() # check

In [6]:
# convert num columns
cols = ['mean_scale_score','level_1_count','level_2_count','level_3_count', 'level_4_count', 'level_4_percentage', 'level_3_4_count',
        'level_1_percentage','level_2_percentage','level_3_percentage','level_4_percentage','level_3_4_percentage']
school_data = change_variable_type(school_data,cols)
# change year to ob
school_data['Year'] = school_data['Year'].astype('object')

In [7]:
school_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 220443 entries, 7 to 626427
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Report Category         220443 non-null  object 
 1   Geographic Subdivision  220443 non-null  object 
 2   school_name             207431 non-null  object 
 3   Grade                   220443 non-null  object 
 4   Year                    220443 non-null  object 
 5   Student Category        220443 non-null  object 
 6   number_tested           220443 non-null  int64  
 7   mean_scale_score        161571 non-null  float64
 8   level_1_count           161571 non-null  float64
 9   level_1_percentage      161571 non-null  float64
 10  level_2_count           161571 non-null  float64
 11  level_2_percentage      161571 non-null  float64
 12  level_3_count           161571 non-null  float64
 13  level_3_percentage      161571 non-null  float64
 14  level_4_count           1